# Admin API

In [1]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
import threading
import logging
from uuid import uuid4

In [113]:
logging.basicConfig()
brokers = "kafka1:9092,kafka2:9093"

In [114]:
conf = {'bootstrap.servers': brokers}

In [115]:
a = AdminClient(conf)

### Creating Topics

In [116]:
topic_names = ["test1p", "test2p", "test3p"]

In [117]:
new_topics = [NewTopic(topic, num_partitions=1, 
        replication_factor=1) for topic in topic_names]

In [118]:
fs = a.create_topics(new_topics)

In [119]:
for topic, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Topic {} created".format(topic))
    except Exception as e:
        print("Failed to create topic {}: {}".format(topic, e))

Failed to create topic test1p: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'test1p' already exists."}
Topic test4p created
Topic test3p created


In [120]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 9 topics:
  "test2p" with 2 partition(s)
  "test4p" with 1 partition(s)
  "_schemas" with 1 partition(s)
  "test22p" with 2 partition(s)
  "test1p" with 1 partition(s)
  "test3p" with 1 partition(s)
  "__consumer_offsets" with 50 partition(s)
  "_confluent-metrics" with 12 partition(s)
  "__confluent.support.metrics" with 1 partition(s)


### Deleting Topics

In [ ]:
fs = a.delete_topics([topic_names[2]], operation_timeout=30)

In [ ]:
for topic, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Topic {} deleted".format(topic))
    except Exception as e:
        print("Failed to delete topic {}: {}".format(topic, e))

### Creating Partitions

In [121]:
topic = topic_names[1]

In [122]:
new_parts = [NewPartitions(topic, int(2))]

In [123]:
fs = a.create_partitions(new_parts, validate_only=False)
# Wait for operation to finish.
for topic, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Additional partitions created for topic {}".format(topic))
    except Exception as e:
        print("Failed to add partitions to topic {}: {}".format(topic, e))

Additional partitions created for topic test4p


In [124]:
md = a.list_topics(timeout=10)
for t in iter(md.topics.values()):
    if str(t)==topic:
        l = t.partitions.values()
        print("Topic {} has {} partitions".format(t, len(l)))
        for p in iter(l):
            if p.error is not None:
                errstr = ": {}".format(p.error)
            else:
                errstr = ""
            print("partition {} leader: {}, replicas: {}," 
                  " isrs: {} errstr: {}".format(p.id, p.leader, p.replicas, p.isrs, errstr))

Topic test4p has 2 partitions
partition 0 leader: 1, replicas: [1], isrs: [1] errstr: 
partition 1 leader: 2, replicas: [2], isrs: [2] errstr: 


## Write to Topic "test1p"

In [126]:
from confluent_kafka import Producer
import sys
conf = {'bootstrap.servers': brokers}

In [127]:
p = Producer(**conf)

In [ ]:
def delivery_callback(err, msg):
        if err:
            sys.stderr.write('%% Message failed delivery: %s\n' % err)
        else:
            sys.stderr.write('%% Message delivered to %s [%d] @ %d\n' %
                             (msg.topic(), msg.partition(), msg.offset()))

In [ ]:
for n in range(1,10):
    try:
        # Produce line (without newline)
        print(n)
        p.produce(topic_names[0], str(n), callback=delivery_callback)
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

In [ ]:
p.poll(0)

In [ ]:
p.flush()

### Reading from Topic test1p

In [ ]:
from confluent_kafka import Consumer, KafkaException

In [ ]:
conf = {
    'bootstrap.servers': brokers, 
    'group.id': str(uuid4()), 
    'session.timeout.ms': 6000,
    'auto.offset.reset': 'earliest'
}

In [ ]:
c = Consumer(conf)

In [ ]:
c.subscribe([topic_names[0]])

In [ ]:
# Read messages from Kafka, print to stdout
try:
    while True:
        msg = c.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        else:
            # Proper message
            sys.stderr.write('%% %s [%d] at offset %d with key %s:\n' %
             (msg.topic(), msg.partition(), msg.offset(), str(msg.key())))
            print(msg.value())
except KeyboardInterrupt:
    sys.stderr.write('%% Aborted by user\n')
finally:
    # Close down consumer to commit final offsets.
    c.close()

## Write to Topic "test2p"

In [128]:
conf = {'bootstrap.servers': brokers}
p = Producer(**conf)

In [129]:
for n in range(1,10):
    try:
        # Produce line (without newline)
        print(n)
        p.produce(topic_names[1], str(n), callback=delivery_callback)
        p.poll(0)
        p.flush()
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

1


<ipython-input-129-f27735018024>:5: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  p.produce(topic_names[1], str(n), callback=delivery_callback)


2
3
4
5
6
7
8
9


% Message delivered to test4p [1] @ 0
% Message delivered to test4p [1] @ 1
% Message delivered to test4p [0] @ 0
% Message delivered to test4p [1] @ 2
% Message delivered to test4p [1] @ 3
% Message delivered to test4p [0] @ 1
% Message delivered to test4p [1] @ 4
% Message delivered to test4p [1] @ 5
% Message delivered to test4p [0] @ 2


In [130]:
conf = {
    'bootstrap.servers': brokers, 
    'group.id': str(uuid4()), 
    'session.timeout.ms': 6000,
    'auto.offset.reset': 'earliest'
}

In [131]:
c = Consumer(conf)
c.subscribe([topic_names[1]])

In [ ]:
# Read messages from Kafka, print to stdout
try:
    while True:
        msg = c.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        else:
            # Proper message
            print("{} [{}] at offset  {} with key  {}:  {}".format(msg.topic(), msg.partition(), msg.offset(), str(msg.key()), str(msg.value())))
except KeyboardInterrupt:
    sys.stderr.write('%% Aborted by user\n')
finally:
    # Close down consumer to commit final offsets.
    c.close()

test4p [0] at offset  0 with key  None:  b'3'
test4p [0] at offset  1 with key  None:  b'6'
test4p [0] at offset  2 with key  None:  b'9'
test4p [1] at offset  0 with key  None:  b'1'
test4p [1] at offset  1 with key  None:  b'2'
test4p [1] at offset  2 with key  None:  b'4'
test4p [1] at offset  3 with key  None:  b'5'
test4p [1] at offset  4 with key  None:  b'7'
test4p [1] at offset  5 with key  None:  b'8'


### What do you notice?